In [1]:
import tomica
print(tomica.__file__)

/Users/zhangyangfa/rust-projects/tomica/tomica/__init__.py


In [2]:
import polars as pl
import random
import time

In [3]:
data = pl.select(
    pl.lit(1).sample(n=1e6, with_replacement=True)  # 先采样
    .map_elements(lambda _: random.uniform(-50, 50), return_dtype=pl.Float64)
    .alias("foo")
).to_series()

In [4]:
def test_speed(fn, window_size):
    st = time.time()
    fn(data, window_size)
    print(f'{fn.__name__:<20} {time.time()-st:>8.4f} s')

In [6]:
for func in tomica.operators.__all__:
    func = getattr(tomica.operators,func)
    test_speed(func,10)

ts_delay               0.0004 s
ts_delta               0.0018 s
ts_returns             0.0027 s
ts_sum                 0.0159 s
ts_product             0.0132 s
ts_max                 0.0161 s
ts_min                 0.0142 s
ts_mean                0.0091 s
ts_std                 0.0177 s
ts_rank                0.0909 s
ts_variance            0.0160 s
ts_quantile_up         0.0299 s
ts_quantile_down       0.0293 s
ts_zscore              0.0273 s
ts_robust_zscore       0.0687 s
